In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Importing Libraries

In [3]:
#statistics
import tensorflow as tf
from tensorflow import keras

#data structures
import numpy as np
import PIL.Image as Image

#network
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
from keras.applications.vgg16 import VGG16

#visualization
import matplotlib.pyplot as plt
from matplotlib import cm as c

#other
from model import create_model
import os
import h5py

print(tf.__version__) #1.14
print(keras.__version__) #2.2.4

/Users/seungmi/anaconda3/envs/pyDL/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/seungmi/anaconda3/envs/pyDL/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/seungmi/anaconda3/envs/pyDL/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/seungmi/anaconda3/envs/pyDL/lib/python3

ModuleNotFoundError: No module named 'model'

In [ ]:
tf.enable_eager_execution()

# Set Hyper Parameters

In [ ]:
batch_size = 100
epochs = 15
IMG_HEIGHT = 255
IMG_WIDTH = 255

# Data Preparation

In [ ]:
PATH = 'ShanghaiTech/'

In [ ]:
# part_a are images of densed crowd
# part_b are images of less densed crowd
part_a = os.path.join(PATH, 'part_A')
part_b = os.path.join(PATH, 'part_B')

In [ ]:
part_a_train = os.path.join(part_a, 'train_data/images/')  
part_b_train = os.path.join(part_b, 'train_data/images/')  
part_a_test = os.path.join(part_a, 'test_data/images/')  
part_b_test = os.path.join(part_b, 'test_data/images/')  

In [ ]:
num_a_tr = len(os.listdir(part_a_train))
num_b_tr = len(os.listdir(part_b_train))

num_a_val = len(os.listdir(part_a_test))
num_b_val = len(os.listdir(part_b_test))

total_train = num_a_tr + num_b_tr
total_val = num_a_val + num_b_val

In [ ]:
print('total training A images:', num_a_tr)
print('total training B images:', num_b_tr)

print('total test A images:', num_a_val)
print('total test B images:', num_b_val)
print("--")
print("Total training images:", total_train)
print("Total test images:", total_val)

In [ ]:
def process_path(file_path):
    label = tf.strings.regex_replace(file_path,"images",'ground-truth')
    label = tf.strings.regex_replace(label, ".jpg",".h5")
    
    return label

In [3]:
def load(image_file):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image, channels = 3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [255, 255])
    
    gt_image = process_path(image_file)

    return image, gt_image

In [4]:
train_dataset = tf.data.Dataset.list_files('ShanghaiTech/*/train_data/images/*.jpg')

In [14]:
train_dataset = train_dataset.map(load, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(400)
train_dataset = train_dataset.batch(batch_size)

In [15]:
test_dataset = tf.data.Dataset.list_files('ShanghaiTech/*/test_data/images/*.jpg')
test_dataset = test_dataset.map(load, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)

# Define functions

In [16]:
loss_object = tf.keras.losses.MeanSquaredError()

In [66]:
def loss_fn(model, input_image, gt_image):

    output = model(input_image)
    pred = np.sum(output) 
    
    gt_file = h5py.File(gt_image.numpy()[0],'r')
    ground_truth = np.asarray(gt_file['density'])
    ground_truth = tf.cast(ground_truth, tf.float32)
    ground_truth = np.sum(ground_truth)
    
    # mean squared error
    loss = loss_object(ground_truth, pred)
    return loss.numpy()

In [74]:
for images, labels in train_dataset.take(1):
    #gt_file = h5py.File(labels.numpy(),'r')
    ground_truth = np.asarray(gt_file['density'])
    ground_truth = tf.cast(ground_truth, tf.float32)
    ground_truth = np.sum(ground_truth)
    print(ground_truth)

TypeError: expected str, bytes or os.PathLike object, not tensorflow.python.framework.ops.EagerTensor

In [18]:
optimizer = Adam(2e-4)

In [23]:
#model = create_model()
from keras.applications.vgg16 import VGG16
model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(255, 255, 3))

In [24]:
model_vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 255, 255, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 255, 255, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 255, 255, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 127, 127, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 127, 127, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 127, 127, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 63, 63, 128)       0     

In [60]:
def generate_images(model, test_input, tar):
    prediction = model(tf.expand_dims(test_input, 0))
    plt.figure(figsize=(15,15))
    tar = h5py.File(tar[0].numpy(),'r')
    tar = np.asarray(tar['density'])
    tar = tf.cast(tar, tf.float32)
    
    display_list = [test_input, tar]#, prediction]
    title = ['Input Image', 'Ground Truth']#, 'Predicted Image']

    print(np.sum(tar))
    print(np.sum(prediction))
    for i in range(2):
        plt.subplot(1, 3, i+1)
        plt.title(title[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
        plt.show()

In [61]:
inp, gt_img = load('ShanghaiTech/part_A/train_data/images/IMG_10.jpg')

In [68]:
#generate_images(model_vgg16, inp, gt_img)

In [67]:
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0
    
    for images, labels in train_dataset:
        grads = grad(model, images, labels)                
        optimizer.apply_gradients(zip(grads, model.variables))
        loss = loss_fn(model, images, labels)
        acc = evaluate(model, images, labels)
        avg_loss = avg_loss + loss
        avg_train_acc = avg_train_acc + acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_dataset:        
        acc = evaluate(model, images, labels)        
        avg_test_acc = avg_test_acc + acc
        test_step += 1    
    avg_test_acc = avg_test_acc / test_step    

    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))
    
    checkpoint.save(file_prefix=checkpoint_prefix)

print('Learning Finished!')

InvalidArgumentError: Invalid reduction dimension (-1 for input with 0 dimension(s) [Op:Mean]